# Large Bayesian vector autorergession models

Up until 2010, most empirical macroeconomic research using vector autoregression (VAR) models was done with small systems (3-5 variables with 1-2 lags). This is because large VARs often contain more parameters than can be fitted by  standard macroeconomic datasets. For example, a $VAR(4)$ with $n = 20$ dependent variables has $1,620$ VAR coefficients, which is much larger than the number of observations in typical quarterly macroeconomic datasets (e.g. forty years of quarterly data (T=160)). As a result, frequentist estimation methods, such as maximum likelihood, are unlikely to have reasonable properties. This is known as the [*over-parameterization problem*](https://en.wikipedia.org/wiki/Overfitting).

In an influential paper, [Banbura, Giannone, and Reichlin (2010)](https://onlinelibrary.wiley.com/doi/full/10.1002/jae.1137?casa_token=RgnCtzbYBjUAAAAA%3AmOIDzTVr46iEoSU73_ZGAmf9RcxmqF20fuv3Ot_jqj8buYLpY0sbERRBU6OGLPauoZZKTMt7-19Niiw) showed that this *over-parameterization* problem can be overcome through the use of informative priors. The basic idea is to use informative priors to push the coefficient values in the VAR towards zero resulting in a system of parsimonious random walks. Such priors are known as *shrinkage priors* (or *regularization priors*) and have the effect of reducing parameter uncertainty, thereby making them better suited than frequentist methods when estimating large VAR models. In their paper, Banbura, Giannone, and Reichlin (2010) not only showed that the use of such priors facilitates the estimation of a VAR with over 100 variables, but also found that this large Bayesian VAR (BVAR) improves upon the forecast performance of commonly used small VARs.

**Remarks**:
1. The prior is imposed *a priori*. If there is a sufficiently strong signal in the data, then *a posteriori*, each equation of the BVAR may follow a more complicated autoregressive process as opposed to a random walk.

## VAR Recap

Recall that the n-variable vector autorergession model of order $p$, denoted VAR(p), is defined as
$$
\begin{equation}
\mathbf{y}_t = \mathbf{b} + \sum_{i=1}^{p}\mathbf{B}_i\mathbf{y}_{t-i} + \boldsymbol{\varepsilon}_t
\end{equation}
$$
where $\mathbf{y}_t,\mathbf{b},\boldsymbol{\varepsilon}_t$ are $n\times 1$ vectors and $\mathbf{B}_i$, $i=1,\dots,p$ are $n\times n$ matrices. 

For estimation purposes, we stack the model over all dates, $t=1,\dots,T$ to get
$$
\mathbf{y} = \mathbf{X}\boldsymbol{\beta} + \boldsymbol{\varepsilon}, \quad \boldsymbol{\varepsilon}\sim \mathcal{N}(\mathbf{0},\mathbf{I}_T\otimes\boldsymbol{\Sigma})
$$
where $\mathbf{y} = (y_1,\dots,y_T)'$ and $\boldsymbol{\varepsilon} = (\varepsilon_1,\dots,\varepsilon_T)'$ are $Tn\times1$,  and 
$\mathbf{X}$ is a $Tn\times nk$ matrix that stacks the regressors into a matrix
$$
\mathbf{X}=\begin{bmatrix}
\mathbf{x}_1 \\ 
\vdots \\ 
\mathbf{x}_T 
\end{bmatrix}
$$

If we are willing to assume prior independence, then the conjugate independent Normal and inverse-Wishart priors facilitate the use of a two block Gibbs sampler. Details are provided in the lecture on VARs.

## Shrinkage priors
A generic Normal distributed prior on the VAR coefficients takes the form 
$$
\boldsymbol{\beta}\sim \mathcal{N}(\boldsymbol{\mu},\mathbf{V}),
$$
where $\boldsymbol{\mu}$ and $\mathbf{V}$ are researcher specified hyperparameters. 

The prior becomes shrinkage prior by setting $\boldsymbol{\mu}=\mathbf{0}$. Set in this manner, the choice of elements of the covariance matrix $ \mathbf{V} $ will govern the degree of shrinkage. 

This means that the *independent Normal and inverse-Wishart prior* that we used to estimate the BVAR in the previous lecture is a shrinkage prior in which all of the BVAR coefficients are pushed towards zero at the same rate. 

More generally, we can use our knowledge of macroeconomic variables to impose some additional structure on the covariance matrix $ \mathbf{V} $. The most common method is the *Minnesota prior*.

### Minnesota prior
Many variants of the Minnesota prior have been proposed (see, e.g., [Karlsson (2013)](https://www.sciencedirect.com/science/article/pii/B9780444627315000154) for
a comprehensive discussion). In each case, it's common to assume that the variance-covariance matrix is diagonal. Hence, there is no relationship among the coefficients of various VAR equations. Here we follow the specification in [Koop and Korobilis (2010)](http://personal.strath.ac.uk/gary.koop/kk3.pdf) which is commonly used in practice. 

For the intercept term, it's common to specify a common scalar variance, denoted $\lambda$. Smaller values imply a more informative prior with more shrinkage, and vice versa.

For the coefficients of the lags, note that the diagonal elements of the prior variance matrix on the lagged coefficient matrices can be written as $ \left(v_1,\dots,v_{nk}\right) = \text{vec}\left(\left(\mathbf{V}_1,\dots,\mathbf{V}_p\right)'\right)$. The $ \left(i,j\right) $-th element of $ \mathbf{V}_r $, $ \mathbf{V}_r^{ij} $, denotes the variance of the $ \left(i,j\right) $-the element of the VAR coefficient matrix $ \mathbf{B}_r $, $ r=1,\dots,p $. 

The Minnesota prior specifies that
$$
\mathbf{V}_r^{ij}=\begin{cases}
\frac{\pi_{1}^{2}}{r^{\pi_{3}}} & \text{for coefficients on own lag }r\text{ for }r=1,\ldots,p,\\
\frac{\pi_{1}^{2}\pi_{2}\sigma_j}{r^{\pi_{3}}\sigma_i} & \text{for coefficients on lag }r\text{ of variable }j\neq i,\text{ for }r=1,\ldots,p,
\end{cases}
$$
where $\pi_{1}, \pi_{2}$ and $\pi_{3}$ are hyperparameters, and $ \sigma_l $ is the standard deviation from an $ AR\left(p\right) $ model for the variable $ l $, $ l=1,\dots,n $. The hyperparameters
1. $\pi_{1}$ controls the overall tightness of the marginal distributions around zero and therefore governs the relative importance of the prior compared to information contained in the data. Smaller values imply a more informative prior with more shrinkage, and vice versa.  
2. $\pi_2$ governs the relative importance of *own-lags* relative to lags of other variables. If $\pi_{2}=1$, then both types of lags are a priori equally important. Conversely, setting $\pi_{2}<1$ implies that own-lags are relatively more important than lags on other variables, and vice-versa.  In most economic applications $\pi_{2}<1$ is a natural choice.
3. $\pi_3$ governs the degree of shrinkage on recent lags relative to distant ones. In most economic applications recent lags are more likely to be important predictors than distant ones making $\pi_{3}\geq 1$ a natural choice. 

To see what the Minnesota prior implies consider a VAR(2) with $n=2$:
$$
\mathbf{V} = 
\begin{bmatrix}
\lambda & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\ 
0 &  \pi_{1}^{2} & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\ 
0 & 0 &  \frac{\pi_{1}^{2}\pi_{2}\sigma_2}{\sigma_1} & 0 & 0 & 0 & 0 & 0 & 0 & 0\\ 
0 & 0 & 0 &  \frac{\pi_{1}^{2}}{2^{\pi_{3}}} & 0 & 0 & 0 & 0 & 0 & 0\\ 
0 & 0 & 0 & 0 &  \frac{\pi_{1}^{2}\pi_{2}\sigma_2}{2^{\pi_{3}}\sigma_1} & 0 & 0 & 0 & 0 & 0\\ 
0 & 0 & 0 & 0 & 0 & \lambda  & 0 & 0 & 0 & 0\\ 
0 & 0 & 0 & 0 & 0 & 0 & \frac{\pi_{1}^{2}\pi_{2}\sigma_1}{\sigma_2} & 0 & 0 & 0\\ 
0 & 0 & 0 & 0 & 0 & 0 & 0 & \pi_{1}^{2} & 0 & 0\\ 
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \frac{\pi_{1}^{2}\pi_{2}\sigma_1}{2^{\pi_{3}}\sigma_2} & 0\\ 
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & \frac{\pi_{1}^{2}}{2^{\pi_{3}}} 
\end{bmatrix}
$$

**Remarks**:
1. **Name**: The Minnesota prior was first proposed in [Litterman (1979)](https://ideas.repec.org/p/fip/fedmwp/115.html), and was subsequently developed by researchers at University of Minnesota [Doan, Litterman, and Sims (1984)](https://www.tandfonline.com/doi/abs/10.1080/07474938408800053) and [Litterman (1986)](https://amstat.tandfonline.com/doi/abs/10.1080/07350015.1986.10509491).
2. **Modern variants**: The original Minnesota prior used [ordinary least squares (OLS)](https://en.wikipedia.org/wiki/Ordinary_least_squares) to estimate the error covariance $\boldsymbol{\Sigma}$ in the VAR model, thereby ignoring parameter uncertainty associated with estimating $\boldsymbol{\Sigma}$. Modern treatments overcome this issue by using the *independent Normal and inverse-Wishart prior* discussed in the previous lecture. No extra estimation tools are required. 
3. **Hyperparameters**: By construction, the Minnesota prior estimates will be sensitive to the choice of hyperparameter values. This is because it's designed to be an informative prior. Following Koop and Korobilis (2010), most practitioners set $\pi_{3}=2$, however values for $\pi_{1}$ and $\pi_{2}$ are generally subjective.
3. **Hierarchical priors**: Dissatisfaction with different user choices of hyperparameter values led to the development of [*hierarchical priors*](https://en.wikipedia.org/wiki/Bayesian_hierarchical_modeling) that introduce priors on the hyperparameters (also called *hyperpriors*) and integrate them out in a Bayesian manner. For instance, [Giannone, Lenza and Primiceri (2015)](https://www.mitpressjournals.org/doi/abs/10.1162/REST_a_00483) show that hyperpriors can generate more accurate macroeconomic forecasts than conventional choices. With this in mind, [Chan, Jacobi and Zhu (2020)](http://joshuachan.org/papers/AD_OptHyper.pdf) have proposed the use of a derivative based approach for efficient selection of the Minnesota hyperparameters.

In [1]:
# Minnesota prior for a BVAR - replace prior for beta in previous codes with the following
# prior mean
pri_beta0 = zeros(nk);

# prior variance
lambda = 10; # overall tightness on intercepts
c1 = 0.2; # overall tightness on lags
c2 = 0.5; # additional shrinkage on cross-lags
c3 = 2;   # rate at which the prior variance decreases with increases lag length

# OLS estimates for standard deviation
sigOLS = zeros(n,1);
for i = 1:n
    yi = y[:,i];
    Xi = [ones(T) X[:, (i+1):n:end ]];
    betai = (Xi'*Xi)\(Xi'*yi);
    e = yi - Xi*betai;
    global sigOLS[i] =  sqrt(mean(e.^2));
end

# Elements of Minneota variance
sig_ratio = sigOLS*transpose(1 ./sigOLS);
C1 = 1 ./transpose(  kron(collect(1:p),ones(n,n)).^c3 );
C2 = repeat(sig_ratio, 1, p);
C3 = c2*ones(n,n);
C3[diagind(C3)] .= 1;
C3 = repeat( C3, 1, p);
vPhi = c1^2 .*C1 .*C2 .*C3;

# Prior covariance and precision matrix
vc0 = lambda*ones(n,1); # variance of intercept terms
vc1 = transpose([vc0 vPhi]);
pri_Vbeta0 = Array(vec(vc1));
pri_invVbeta0 = sparse(diagm(1 ./pri_Vbeta0));

LoadError: UndefVarError: nk not defined

# Conclusion
VARs tend to be highly parameterized, and Bayesian methods allow us to overcome this overparameterization problem through the use of informative priors, known as shrinkage priors. The most prominent of these is the Minnesota prior which captures the following ideas
1. Own lags are more likely to be important predictors than lags of other variables
2. More recent lags are more likely to be important predictors than distant ones

To incorporate the Minnesota prior beliefs into the BVAR, simply change the prior for beta using the code provided, and then estimate the BVAR using the code made available for the idependent Normal and inverse-Wishart prior.

## Recent research
Over the past few years, researchers have proposed *adaptive hierarchical shrinkage priors* which have shown to have optimal theoretical properties when applied to sparse datasets. The main idea is to leave preserve the signal from 'large' coefficients while strongly shrinking 'small' coefficients  to zero. Popular examples include the *Dirichlet-Laplace* ([Kastner and Huber, 2020](https://onlinelibrary.wiley.com/doi/full/10.1002/for.2680)),  *Horseshoe* ([Follett and Yu, 2017](https://arxiv.org/abs/1709.07524)) and *Normal-Gamma* ([Huber and Feldkircher, 2019](https://www.tandfonline.com/doi/full/10.1080/07350015.2016.1256217)) priors, which are from the family of *global-local priors* ([Polson and Scott, 2010](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.180.727&rep=rep1&type=pdf)). 

Despite having good theoretical properties, [Cross, Poon and Hou (2020)](https://www.sciencedirect.com/science/article/pii/S0169207019302547) find that these adaptive hierarchical priors don't seem to forecast better than a variant of the Minnesota prior where the hyperparameters are selected based on the data. A possible reason for this result is that macroeconomic datasets are not *sparse* but *dense*, a result that was originally claimed in [Giannoni, Lenza and Primiceri (2018)](https://ideas.repec.org/p/fip/fednls/87258.html). This has lead to new research developments on Minnesota-type adaptive hierarchical priors ([Chan, 2020](http://joshuachan.org/papers/BVAR-MAHP.pdf)) and hierarchical priors for time-varying parameter models ([Huber, Koop and Onorante, 2020](https://www.tandfonline.com/doi/full/10.1080/07350015.2020.1713796)), and remains an area of active research. 

## Recommended reading
1. For those interested in learning more about large BVARs, I recommend reading the manuscript by [Koop and Korobilis (2010)](http://personal.strath.ac.uk/gary.koop/kk3.pdf) and [Large Bayesian Vector Autoregressions](http://joshuachan.org/papers/large_BVAR.pdf) by Joshua CC Chan, in [Macroeconomic Forecasting in the Era of Big Data: Theory and Practice
](https://www.springer.com/gp/book/9783030311490).
2. For those interested in estimating large BVARs with non-Gaussian, heteroscedastic and serially dependent errors, see [Chan (2020)](http://www.joshuachan.org/papers/BVAR.pdf).